ic10
https://pypi.org/project/icd10-cm/

ehalthKD 2020
https://knowledge-learning.github.io/ehealthkd-2020/


ehealthKD ner 2019
https://github.com/turmovasiring/eHEALTHKD


Codiesp 2020
https://temu.bsc.es/codiesp/

clef 19
https://clefehealth.imag.fr/?page_id=171

wbi
https://github.com/turmovasiring/wbi-clef19/blob/master/bert_multilabel.py

mlp
https://github.com/turmovasiring/multilabel-classification-bert-icd10/blob/master/models.py
https://colab.research.google.com/drive/1iPrvx0WtFILYrUbCmVS6LA1RUud2_rjt?authuser=1#scrollTo=xc-JR7S5GHp1

isa
https://github.com/isegura?tab=repositories

isa multilabel
https://github.com/isegura/MultiLabel/tree/master/git-multilabel/papers


link
https://docs.google.com/document/d/1SDnvcR8xertFnk-aonplUcFn5gO-VBgXqVdrXxNdGQI/edit


NER with CRF
https://github.com/Akshayc1/named-entity-recognition
https://www.aitimejournal.com/@akshay.chavan/complete-tutorial-on-named-entity-recognition-ner-using-python-and-keras



Exercise_Spacy.ipynb (clase 3)
NER_by_CRF.ipynb (Clase 6)   hay muchos
Normalization
Stopwords
Vectorization
Word embedding
TExt similarity
text simplification
clasification



ICD10-CM

https://www.nlm.nih.gov/research/umls/sourcereleasedocs/current/ICD10CM/index.html

ICD10-PCS
https://www.nlm.nih.gov/research/umls/sourcereleasedocs/current/ICD10PCS/index.html



CIE-10-CM
https://eciemaps.mscbs.gob.es/ecieMaps/browser/index_10_mc.html

CIE-10-PCS
https://eciemaps.mscbs.gob.es/ecieMaps/browser/index_10_pcs.html

Descargar test text files (no annotations) 2 march
https://temu.bsc.es/codiesp/

## Connect Drive


In [0]:
from google.colab import drive
drive.mount("/content/drive/")
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
drive  sample_data


# Import libraries

In [0]:
#dataframes
import pandas as pd
from tabulate import tabulate

#regex
import re

#models
import spacy

In the notebook **"0904_terminado_Crear tsv a partir de text files.ipynb"**, there is a section called **"Create a tsv from all text files (train and dev)"**. It is necessary to execute it before load the independent text files.

https://colab.research.google.com/drive/1JhAqsmZBLJzpb0lA8BlU2Kzldv6WQ0b5#scrollTo=DLKJlTsxQ4YB


#Load data

1. Articles, Diagnostics/Procedures, train
2. Articles, Diagnostics/Procedures, dev
3. Codes, Diagnostics, train/dev
4. Annotations, Diagnostics, train
5. Annotations, Diagnostics, dev
6. Codes, Procedures, train/dev
7. Annotations, Procedures, train
8. Annotations, Procedures, dev

##Load articles from tsv (train and test)

In [0]:
def load_articles(path):
  df = pd.read_csv(path, sep=",", engine="python", encoding='utf-8')
  print("--- Loaded dataset:", path)

  print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(df.head())
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))

  return df

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'

print("- Load 1. Articles, Diagnostics/Procedures, train")
train_file_articles = path_codiesp + 'train_dev/train/train_articles_codiesp.csv'
df_train_articles = load_articles(train_file_articles)

print("- Load 2. Articles, Diagnostics/Procedures, dev")
dev_file_articles = path_codiesp + 'train_dev/dev/dev_articles_codiesp.csv'
df_dev_articles = load_articles(dev_file_articles)

- Load 1. Articles, Diagnostics/Procedures, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_articles_codiesp.csv
--- Number of rows is 500 x number of columns 2
- Load 2. Articles, Diagnostics/Procedures, dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/dev/dev_articles_codiesp.csv
--- Number of rows is 250 x number of columns 2


##Loading annotations and codes from tsv (train and dev, D and P)

In [0]:
def load_data(path, names):
  df = pd.read_csv(path, sep="\t", names=names)
  print("--- Loaded dataset:", path)

  print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(df.head())
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))

  return df

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'
names_annotations=["ArticleID", "Label", "ICD10-code", "Text-reference"]  #Columns: articleID \t label \t ICD10-code \t text-reference
names_codeD=["Code", "Es-description", "En-description"]   # Columns: code \t es-description \t en-description
names_codeP=["Code", "Es-description", "En-description", "Approach?"]   # Columns: code \t es-description \t en-description \t approach?

print("- Load 3. Codes, Diagnostics, train/dev")
file_codeD = path_codiesp + 'codiesp_codes/codiesp-D_codes.tsv'
df_codeD = load_data(file_codeD, names_codeD)

print("- Load 4. Annotations, Diagnostics, train")
file_train_annotationsD = path_codiesp + 'train_dev/train/train_annotations_codiespD_processed.tsv'
df_train_annotationsD = load_data(file_train_annotationsD, names_annotations)

print("- Load 5. Annotations, Diagnostics, dev")
file_dev_annotationsD = path_codiesp + 'train_dev/dev/dev_annotations_codiespD_processed.tsv'
df_dev_annotationsD = load_data(file_dev_annotationsD, names_annotations)

print("- Load 6. Codes, Procedures, train/dev")
file_codeP = path_codiesp + 'codiesp_codes/codiesp-P_codes.tsv'
df_codeP = load_data(file_codeP, names_codeP)

print("- Load 7. Annotations, Procedures, train")
file_train_annotationsP = path_codiesp + 'train_dev/train/train_annotations_codiespP_processed.tsv'
df_train_annotationsP = load_data(file_train_annotationsP, names_annotations)

print("- Load 8. Annotations, Procedures, dev")
file_dev_annotationsP = path_codiesp + 'train_dev/dev/dev_annotations_codiespP_processed.tsv'
df_dev_annotationsP = load_data(file_dev_annotationsP, names_annotations)

- Load 3. Codes, Diagnostics, train/dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/codiesp_codes/codiesp-D_codes.tsv
--- Number of rows is 71486 x number of columns 3
- Load 4. Annotations, Diagnostics, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_annotations_codiespD_processed.tsv
--- Number of rows is 7524 x number of columns 4
- Load 5. Annotations, Diagnostics, dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/dev/dev_annotations_codiespD_processed.tsv
--- Number of rows is 3589 x number of columns 4
- Load 6. Codes, Procedures, train/dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/codiesp_codes/codiesp-P_codes.tsv
--- Number of rows is 87170 x number of columns 4
- Load 7. Annotations, Procedures, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_annot

##Print dataframes for all the data

1. Articles, Diagnostics/Procedures, train
2. Articles, Diagnostics/Procedures, dev
3. Codes, Diagnostics, train/dev
4. Annotations, Diagnostics, train
5. Annotations, Diagnostics, dev
6. Codes, Procedures, train/dev
7. Annotations, Procedures, train
8. Annotations, Procedures, dev



In [0]:
print("- Print 1. Articles, Diagnostics/Procedures, train")
print(tabulate(df_train_articles.head(), headers='keys', tablefmt='psql'))

print("- Print 2. Articles, Diagnostics/Procedures, dev")
print(tabulate(df_dev_articles.head(), headers='keys', tablefmt='psql'))

print("- Print 3. Codes, Diagnostics, train/dev")
print(tabulate(df_codeD.head(), headers='keys', tablefmt='psql'))

print("- Print 4. Annotations, Diagnostics, train")
print(tabulate(df_train_annotationsD.head(), headers='keys', tablefmt='psql'))

print("- Print 5. Annotations, Diagnostics, dev")
print(tabulate(df_dev_annotationsD.head(), headers='keys', tablefmt='psql'))

print("- Print 6. Codes, Procedures, train/dev")
print(tabulate(df_codeP.head(), headers='keys', tablefmt='psql')) 

print("- Print 7. Annotations, Procedures, train")
print(tabulate(df_train_annotationsP.head(), headers='keys', tablefmt='psql'))   

print("- Print 8. Annotations, Procedures, dev")
print(tabulate(df_dev_annotationsP.head(), headers='keys', tablefmt='psql'))

- Print 1. Articles, Diagnostics/Procedures, train
+----+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Spacy functions

Spacy is a Python library that allows us to perform many NLP tasks such as
 
 
    - Tokenization
    - Pos Tagging
    - Entity Detection
    - Dependency Parsing
    - Semantic Similarity

 
 Another important advantages of  Spacy is that it can be used to generated word vectors (word embeddings) for every word in a sentence. Spacy already includes some of these pre-trained models, which you can use right now!!.
 

 https://nlpforhackers.io/complete-guide-to-spacy/
 
  

Spacy token attributes https://spacy.io/api/token

Spacy NER https://spacy.io/usage/linguistic-features

###Install Spacy and the model in spanish.

Using displaCy in Jupyter notebooks. Reference in : https://spacy.io/usage/visualizers

For dictionary based NER, it is necessary spacy-lookup library.

spacy-lookup library:   Github.

https://github.com/mpuig/spacy-lookup

Notebook.

https://colab.research.google.com/drive/1Xsne2tMz2IDk0X2Gn-9GIUcVDpAfS3-W?authuser=1#scrollTo=MR_FqfiQ5cLx


In [0]:
#pip install -U spacy #spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)   1 time
#!pip install spacy
!python -m spacy download es          #Every time it reload    ,   Successfully installed es-core-news-sm-2.2.5

import spacy
nlp = spacy.load('es')
#nlp = spacy.load("en_core_web_sm")  #Do es not work
print('spacy.es loaded')
  
!pip install spacy-lookup    #For Dictionary based NER ,  Successfully installed flashtext-2.7 spacy-lookup-0.1.0
from spacy_lookup import Entity

from spacy import displacy


     |████████████████████████████████| 16.2MB 1.2MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=d43056c4ad35c2a268c23c88a02d26099c7f94a08535a2e2f261462b9641d300
  Stored in directory: /tmp/pip-ephem-wheel-cache-tpd8r2ny/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')
spacy.es loaded
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9299 sha256=1bb4cd9c073be6d6cda3e226431eef46ad12c3d15a4c63a38c77083a6c5165c2
  Stored in directory: /root/.cache/pip/wheels/37/db/d7/fe74f7cb8e5c3afed90fe6f4967c933a6f13d81ab6b3d3128c
Successfully built flashtext


#Automatic NER (not necessary)

### NER default


In [0]:
document = nlp(text_train)

for i,s in enumerate(document.sents):             #divide doc into sents
  print("---------------------------------------------------------------------------------------------------------------------------")
  print("Sentence text: ", i, s)
  #print('Named entities for the sentence: ', i)  
  start_sentence_text = s.start_char
  #print("start_sentence_text:", start_sentence_text)
  end_sentence_text = s.end_char
  #print("end_sentence_text:", end_sentence_text)
  
  for e in s.ents:                                    #divide senteces into entities
    start_word_text = e.start_char    #entity begin respect with document
    #print("start_word_text:", start_word_text)
    end_word_text = e.end_char        #entity end respect with document
    #print("end_word_text:", end_word_text)
    start_word_sentence = start_word_text - start_sentence_text    #entity begin respect with sentence
    end_word_sentence = end_word_text - start_sentence_text 
    print('\t',"Entity text reference:", e.string,"\t---  start_word_sentence:", start_word_sentence, "\t---  end_word_sentence:",  end_word_sentence)


NameError: ignored

###NER accesing entities (iob, not necessary)

In [0]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(text_train)

# document level
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(ents)

# token level
ent_1 = [doc[0].text, doc[0].ent_iob_, doc[0].ent_type_]
#ent_francisco = [doc[1].text, doc[1].ent_iob_, doc[1].ent_type_]
print(ent_1)  # ['San', 'B', 'GPE']
#print(ent_francisco)  # ['Francisco', 'I', 'GPE']

In [0]:
import spacy
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")
doc = nlp("fb is hiring a new vice president of global policy San Francisco")
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('Before', ents)
# the model didn't recognise "fb" as an entity :(

fb_ent = Span(doc, 0, 1, label="ORG") # create a Span for the new entity
doc.ents = list(doc.ents) + [fb_ent]

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('After', ents)
# [('fb', 0, 2, 'ORG')] 🎉

In [0]:
import numpy
import spacy
from spacy.attrs import ENT_IOB, ENT_TYPE

nlp = spacy.load("en_core_web_sm")
doc = nlp.make_doc("London is a big city in the United Kingdom. fb")
print("Before", doc.ents)  # []

header = [ENT_IOB, ENT_TYPE]
attr_array = numpy.zeros((len(doc), len(header)))
attr_array[0, 0] = 3  # B
attr_array[0, 1] = doc.vocab.strings["GPE"]
#doc.from_array(header, attr_array)
#print("After", doc.ents)  # [London]

###NER extraction with Displacy (Dictionary based using entities from AnnotationsD and AnnotationsP, only train)


Using displaCy in Jupyter notebooks. Reference in : https://spacy.io/usage/visualizers

For dictionary based NER, it is necessary spacy-lookup library.

spacy-lookup library:   Github.

https://github.com/mpuig/spacy-lookup

Notebook.

https://colab.research.google.com/drive/1Xsne2tMz2IDk0X2Gn-9GIUcVDpAfS3-W?authuser=1#scrollTo=MR_FqfiQ5cLx


https://spacy.io/usage/spacy-101#features

https://spacy.io/usage/linguistic-features#named-entities

https://spacy.io/usage/training#ner


EntityRecognizer: recognizes entities
https://spacy.io/api/entityrecognizer

before 

EntityLinker: resolves named entities to knoledge based IDs
https://spacy.io/api/entitylinker


Bad ner tutorial
https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718

Phrase matcher
https://spacy.io/api/phrasematcher


In [0]:
def load_dictionary(df):
  """This function reads a df and save the fourth column into a list. 
  This list is an input parameter, which is modified. We need to pass
  the list as parameter, because if the list is returned, 
  its type becomes to NoneType"""
  dictionary=[]
  for index, row in df.iterrows():
    dictionary.append(row[3].lower())
  print(dictionary)
  print('length of the dictionary loaded: ', len(dictionary))

  return(dictionary)

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'

print("---Load dictionary 4. Annotations, Diagnostics, train")
train_diagnostics_dictionary = load_dictionary(df_train_annotationsD)
#print(train_diagnostics)

#print("---Load dictionary 5. Annotations, Diagnostics, dev")
#dev_diagnostics_dictionary = load_dictionary(df_dev_annotationsD)
#print(dev_diagnostics)

print("---Load dictionaty 7. Annotations, Procedures, train")
train_procedures_dictionary = load_dictionary(df_train_annotationsP)
#print(train_procedures)

#print("---Load dictionaty 8. Annotations, Procedures, dev")
#dev_procedures_dictionary = load_dictionary(df_dev_annotationsP)
#print(dev_procedures)

-  **Add Named Entities metadata to Doc objects in Spacy.** First, we load the model and replace the NER module with de entity diagnosticEnt. We do this to avoid overlapping of entities. Then, we also add procedureEnt.

In [0]:
diagnosticEnt = Entity(keywords_list=train_diagnostics_dictionary,label="DIAGNOSTICO")
print(type(diagnosticEnt))

procedureEnt = Entity(keywords_list=train_procedures_dictionary,label="PROCEDIMIENTO")
print(type(procedureEnt))

#We replace the common entities with diagnostics
nlp.replace_pipe("ner", diagnosticEnt)
#We add pipe
nlp.add_pipe(procedureEnt, last=True)

print('entities loaded in nlp')

Pipeline name: ['tagger', 'parser', 'ner', 'entity']

- **Detect Named Entities using dictionaries.** We can process a text and show its entities.

In [0]:
ents=["DIAGNOSTICO","PROCEDIMIENTO"]
colors={"DIAGNOSTICO":"#9FDEF9","PROCEDIMIENTO":"#BA9FF9"}    #blue and purple
options = {"ents": ents, "colors": colors}



document = nlp(text)


#dict([(str(x), x.label_) for x in nlp(str(text)).ents])  #dictionary with entities and types {'Brucella': 'DIAGNOSTICO',... 'varices': 'DIAGNOSTICO'}

document = nlp(text)
for i, sent in enumerate(document.sents):      #divide document into sentences
    #print(i, sent.text)
    #print('Original Sentence: ', sent.text)
    print(sent.start_char, sent.end_char)    #start_char and end_char are the position of the first and last character of sentence respecting the document.
    #for entity in doc.ents:                        #divide document into entities
      #print('Type: {}, Value: {}, star: {}, end: {}'.format(entity.label_, entity.text,entity.start_char, entity.end_char)  #start_char and end_char are the position of the first and last character of an entity, respect the document begining.  
    doc = nlp(sent.text)
    displacy.render(doc, jupyter=True, style="ent", options=options)
    #for token in sent:                        #divide sentences into tokens
      #print('\t',token.orth_, token.pos_)   #	ID of the verbatim text content, 	Coarse-grained part-of-speech.
    print("----------------------------------------------------------------------------------------------------------------------------")

###NER evaluation over dev

In [0]:
ents=["DIAGNOSTICO","PROCEDIMIENTO"]
colors={"DIAGNOSTICO":"#9FDEF9","PROCEDIMIENTO":"#BA9FF9"}    #blue and purple
options = {"ents": ents, "colors": colors}


#dict([(str(x), x.label_) for x in nlp(str(text)).ents])  #dictionary with entities and types {'Brucella': 'DIAGNOSTICO',... 'varices': 'DIAGNOSTICO'}

document = nlp(text)
for i, sent in enumerate(document.sents):      #divide document into sentences
    #print(i, sent.text)
    #print('Original Sentence: ', sent.text)
    print(sent.start_char, sent.end_char)    #start_char and end_char are the position of the first and last character of sentence respecting the document.
    #for entity in doc.ents:                        #divide document into entities
      #print('Type: {}, Value: {}, star: {}, end: {}'.format(entity.label_, entity.text,entity.start_char, entity.end_char)  #start_char and end_char are the position of the first and last character of an entity, respect the document begining.  
    doc = nlp(sent.text)
    displacy.render(doc, jupyter=True, style="ent", options=options)
    #for token in sent:                        #divide sentences into tokens
      #print('\t',token.orth_, token.pos_)   #	ID of the verbatim text content, 	Coarse-grained part-of-speech.
    print("----------------------------------------------------------------------------------------------------------------------------")